In [1]:
import pandas as pd
import numpy as np
from hmmlearn import hmm

## Baum -Welch algorithm

source http://www.adeveloperdiary.com/data-science/machine-learning/derivation-and-implementation-of-baum-welch-algorithm-for-hidden-markov-model/

In [2]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
 
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1] @ a[:, j] * b[j, V[t]]
 
    return alpha

In [3]:
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]) @ a[j, :]
 
    return beta

In [4]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)

    for n in range(n_iter):
        ###estimation step
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            # joint probab of observed data up to time t @ transition prob * emisssion prob as t+1 @
            # joint probab of observed data from time t+1
            denominator = (alpha[t, :].T @ a * b[:, V[t + 1]].T) @ beta[t + 1, :]
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        ### maximization step
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))

    return a, b

## Usage

In [5]:
data = pd.read_csv('data_python.csv.txt')

V = data['Visible'].values

# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))

n_iter = 100
a_model, b_model = baum_welch(V.copy(), a.copy(), b.copy(), initial_distribution.copy(), n_iter=n_iter)
print(f'Custom model A is \n{a_model} \n \nCustom model B is \n{b_model}')

Custom model A is 
[[0.53816345 0.46183655]
 [0.48664443 0.51335557]] 
 
Custom model B is 
[[0.16277513 0.26258073 0.57464414]
 [0.2514996  0.27780971 0.47069069]]


In [8]:
# compore to hmmlearn
model = hmm.MultinomialHMM(n_components=2, n_iter=n_iter, init_params="")
model.startprob_ = initial_distribution
model.transmat_ = a
model.emissionprob_ = b

model.fit([V])
print(f'hmmlearn A \n{model.transmat_}')
print(f'hmmlearn B \n{model.emissionprob_}')

hmmlearn A 
[[0.49678881 0.50321119]
 [0.49348432 0.50651568]]
hmmlearn B 
[[0.15912386 0.2763461  0.56453004]
 [0.25179386 0.26380042 0.48440572]]


In [9]:
print(f'A difference example implementation and hmmlearn \n{a_model - (model.transmat_)}')
print(f'B difference example implementation and hmmlearn \n{b_model - (model.emissionprob_)}')
print(f'\nA example implementation and hmmlearn allclose: {np.allclose(a_model, model.transmat_, atol=0.1)}')
print(f'B example implementation and hmmlearn allclose: {np.allclose(b_model, model.emissionprob_, atol=0.1)}')

A difference example implementation and hmmlearn 
[[ 0.04137464 -0.04137464]
 [-0.00683989  0.00683989]]
B difference example implementation and hmmlearn 
[[ 0.00365127 -0.01376537  0.0101141 ]
 [-0.00029426  0.01400929 -0.01371503]]

A example implementation and hmmlearn allclose: True
B example implementation and hmmlearn allclose: True
